In [1]:
from dataset_UNSW_NB15 import Dataset_UNSW_NB15
from dataset_NSL_KDD import Dataset_NSL_KDD
from dataset_IoT_Network_Intrusion_Dataset import Dataset_IoT_Network_Intrusion_Dataset
from experiment import Experiment
import matplotlib.pyplot as plt


In [ ]:
dataset1 = Dataset_NSL_KDD(config_filename="config_NSL_KDD.json")
exp1 = Experiment(dataset1.data, dataset1.labels, dataset1.config)
exp1.run()

In [ ]:
dataset2 = Dataset_UNSW_NB15(config_filename="config_UNSW_NB15.json")
exp2 = Experiment(dataset2.data, dataset2.labels, dataset2.config)
exp2.run()

In [2]:
dataset3 = Dataset_IoT_Network_Intrusion_Dataset(config_filename="config_IoT_Network_Intrusion_Dataset.json")
exp3 = Experiment(dataset3.data, dataset3.labels, dataset3.config)
exp3.run()

ValueError: could not convert string to float: '192.168.0.1-192.168.0.14-80-53168-6'

In [ ]:
# kの値と対応する精度を保存するリスト
k_values = []
accuracies = []
dataset2 = Dataset_UNSW_NB15(config_filename="config_UNSW_NB15.json")
exp2 = Experiment(dataset2.data, dataset2.labels, dataset2.config)

# kの値を10から500まで変えて実験
for k in range(50, 2000, 50):
    print(f"Current k: {k}", end="\r")
    # モデルの訓練と評価
    exp2.run(model_params={"k": k , "n_features" : 30})
    
    # kの値と精度を保存
    k_values.append(k)
    accuracies.append(exp2.accuracy)

# 精度をグラフで表示
plt.plot(k_values, accuracies)
plt.xlabel('k')
plt.ylabel('Accuracy')
plt.title('Accuracy for different values of k')
plt.show()